In [2]:
import requests
import pandas as pd

In [60]:
from datetime import datetime, timedelta


'2023-09-28'

In [61]:
def get_chemrxiv(days_back):
    skip_num = 0
    total_count=1
    start_date = (datetime.now()-timedelta(days=days_back)).strftime('%Y-%m-%d')
    def get_affiliation(x):
        out = []
        for affil in x:
            out.append(affil['name'])
        return out
    def get_authors_affils(x):
        authors = []
        affiliations = []
        for name in x:
            authors.append(name['firstName']+' '+name['lastName'])
            affiliations.extend(get_affiliation(name['institutions']))
        return authors, list(set(affiliations))
    def get_chemrixiv_subjects(x):
        subjects = []
        for subj in x:
            subjects.append(subj['name'])
        return subjects
    out = []
    while skip_num < total_count:
        response = requests.get(f'https://chemrxiv.org/engage/chemrxiv/public-api/v1/items?limit=50&skip={skip_num}&searchDateFrom={start_date}T00:00:00.000Z')
        json_resp = response.json()
        total_count = json_resp['totalCount']
        skip_num+=len(json_resp['itemHits'])
        print(skip_num)
        for paper in json_resp['itemHits']:
            paper_info = {}
            paper_info['paper_id'] = paper['item']['id']
            paper_info['authors'],paper_info['affiliations'] = get_authors_affils(x = paper['item']['authors'])
            paper_info['subjects'] = get_chemrixiv_subjects(paper['item']['categories'])
            paper_info['title'] = paper['item']['title']
            paper_info['abstract'] = paper['item']['abstract']
            paper_info['url'] = 'https://chemrxiv.org/engage/chemrxiv/article-details/'+str(paper['item']['id'])
            paper_info['date'] = paper['item']['publishedDate'][:10]
            paper_info['source'] = paper['item']['origin']
            out.append(paper_info)
    return pd.DataFrame.from_records(out)

In [62]:
get_chemrxiv(3)

50
76


,paper_id,authors,affiliations,subjects,title,abstract,url,date,source
0,6516c4f8a69febde9eeb8ad2,"[Piermarco Saraceno, Vladislav Sláma, Lorenzo ...",[Dipartimento di Chimica e Chimica Industriale...,"[Theoretical and Computational Chemistry, Phys...",First-principles simulation of excitation ener...,Abstract \tThe dynamics of delocalized exciton...,https://chemrxiv.org/engage/chemrxiv/article-d...,2023-09-29,CHEMRXIV
1,65168004ade1178b24567cd3,"[Rıza Özçelik, Sarah de Ruiter, Francesca Gris...","[Centre for Living Technologies, Alliance TU/e...","[Theoretical and Computational Chemistry, Biol...",Structured State-Space Sequence Models for De ...,De novo drug design has witnessed remarkable p...,https://chemrxiv.org/engage/chemrxiv/article-d...,2023-09-29,CHEMRXIV
2,6516843fade1178b2456a2f2,"[Diana Lamaa, Catherine Cailleau, Juliette Ve...","[Institut de Chimie des Substances Naturelles,...","[Biological and Medicinal Chemistry, Nanoscience]",Overcoming Solubility Challenges: Liposomal is...,This study evaluated the potential of isoCoQ-C...,https://chemrxiv.org/engage/chemrxiv/article-d...,2023-09-29,CHEMRXIV
3,6516778ea69febde9ee728ef,"[Javed Y. Shaikh, Anindita Bhowmick, Abhijit...",[Indian Institute of Science Education and Res...,"[Organic Chemistry, Catalysis, Homogeneous Cat...",Palladium Catalyzed C3-(sp2)-H Alkenylation of...,A Pd (II)-catalyzed direct C3-(sp2)-H alkenyla...,https://chemrxiv.org/engage/chemrxiv/article-d...,2023-09-29,CHEMRXIV
4,65161cfa0065940912145855,"[Haoxin Ye, Xinzhe Zheng, Haoming Yang, Matt...","[University of British Columbia, University of...","[Analytical Chemistry, Nanoscience, Agricultur...",Rapid Detection of Micro/Nanoplastics Via Inte...,The fact that there is an accumulation of micr...,https://chemrxiv.org/engage/chemrxiv/article-d...,2023-09-29,CHEMRXIV
...,...,...,...,...,...,...,...,...,...
71,6512b162ade1178b2424c325,[julio coll],[CSIC],"[Biological and Medicinal Chemistry, Bioinform...",Exploring non-toxic co-evolutionary docking,Drug-spaces of nine crystallographic protein /...,https://chemrxiv.org/engage/chemrxiv/article-d...,2023-09-27,CHEMRXIV
72,65127c57ade1178b24220d4e,"[Yu Takatani, Yuto Ohata, Takato Inoue, Naoki ...","[Laboratory of Chemical Ecology, Department of...","[Organic Chemistry, Analytical Chemistry, Agri...",Identification of (+)-7R-Actinidine and its Bi...,"Actinidine, a terpene alkaloid, is commonly fo...",https://chemrxiv.org/engage/chemrxiv/article-d...,2023-09-27,CHEMRXIV
73,65127c3eade1178b24220b5d,"[Michael Y. Toriyama, Cheng-Wei Lee, G. Jeffre...","[Colorado School of Mines, Northwestern Univer...","[Theoretical and Computational Chemistry, Mate...",Defect Chemistry and Doping of Lead Phosphate ...,Lead phosphate oxo apatite Pb10(PO4)6O is clai...,https://chemrxiv.org/engage/chemrxiv/article-d...,2023-09-27,CHEMRXIV
74,65130f76ade1178b2429cd68,"[Raul Ortega Ochoa, Bardi Benediktsson, Rena...",[Technical University of Denmark],"[Theoretical and Computational Chemistry, Phys...",Materials Funnel 2.0 - Data-driven hierarchica...,Innovating ways to explore the materials phase...,https://chemrxiv.org/engage/chemrxiv/article-d...,2023-09-27,CHEMRXIV


### This is a test for Scopus data pull

In [64]:
df_explored = pd.read_pickle('last_month_wo_llama.pkl')
explored_files = df_explored[df_explored['score_divider'] <=2]['paper_id'].values.tolist()#if a paper is for some reason updated then catch it otherwise ignore as we have already indexed it
explored_files = dict(zip(explored_files, [None]*len(explored_files)))

In [67]:
from pybliometrics.scopus import AbstractRetrieval
def get_data_per_doi(doi, output_dict):
    ab = AbstractRetrieval(doi, view='FULL')
    output_dict['title'] = ab.title
    abstract = ab.abstract
    if not abstract:
        abstract = ab.description
    output_dict['abstract'] =abstract
    output_dict['url']= ab.scopus_link
    subjects =[]
    for area in ab.subject_areas:
        subjects.append(area.area)
    authors = []
    affiliations = []
    for author in ab.authors:
        authors.append(author.indexed_name)
    affiliations = []
    for affiliation in ab.affiliation:
        affiliations.append(affiliation.name)
    output_dict['affiliations'] = affiliations
    output_dict['authors'] = authors
    output_dict['subjects'] = subjects
    return output_dict
def get_scopus(days_back):
    api_key = 'f7401115c3b98c2edae8c1203c2f15b4'
    url = 'https://api.elsevier.com/content/search/scopus'

    # Define the query parameters
    params = {
        'apiKey':api_key,
        'query': f'LANGUAGE(english) AND AFFILCOUNTRY(United States) AND (AFFILORG(Georgia) OR AFFILORG(Emory)) AND date={str(datetime.today().year)}',
        'count': 100,  # You can adjust the number of results per page as needed
    }
    out =[]
    dates_back_found = True
    while dates_back_found:
        response = requests.get(url, params=params)
        response_json = response.json()['search-results']['entry']
        print('scopus::::::')
        print(len(response_json))
        for i in range(len(response_json)):
            paper_dict = {}
            paper_dict['paper_id'] = response_json[i]['dc:identifier']
            if paper_dict['paper_id'] in explored_files:
                print('passing:  '+str(paper_dict['paper_id']))
                continue
            paper_dict['date'] = response_json[i]['prism:coverDate']
            paper_dict = get_data_per_doi(response_json[i]['prism:doi'], paper_dict)
            out.append(paper_dict)
            if datetime.strptime(response_json[i]['prism:coverDate'], '%Y-%m-%d') < datetime.today()-timedelta(days=days_back):
                dates_back_found = False
            
        next_dict = response.json()['search-results']['link'][2]
        if next_dict['@ref'] =='next':
            url = next_dict['@href']
        params = {
            'apiKey':api_key,
            'count': 100,  # You can adjust the number of results per page as needed
        }
    df = pd.DataFrame.from_records(out)
    df['source'] = 'Scopus'
    return df
df = get_scopus(2)
df

scopus::::::
100


,paper_id,date,title,abstract,url,affiliations,authors,subjects,source
0,SCOPUS_ID:85171809400,2023-12-01,Cluster analysis driven by unsupervised latent...,"© 2023, Springer Nature Limited.Unsupervised c...",https://www.scopus.com/inward/record.uri?partn...,"[University of Georgia, Georgia Institute of T...","[Sikora A., Jeong H., Yu M., Chen X., Murray B...",[Multidisciplinary],Scopus
1,SCOPUS_ID:85171388196,2023-12-01,"Type, dose, and outcomes of physical therapy i...","© 2023, BioMed Central Ltd., part of Springer ...",https://www.scopus.com/inward/record.uri?partn...,"[Quillen-Dishner College of Medicine, Universi...","[Hoppes C.W., Anson E.R., Carender W.J., March...",[Medicine (miscellaneous)],Scopus
2,SCOPUS_ID:85170350092,2023-12-01,Mitigating underreported error in food frequen...,"© 2023, BioMed Central Ltd., part of Springer ...",https://www.scopus.com/inward/record.uri?partn...,"[Rollins School of Public Health, Woodruff Hea...","[Popoola A.A., Frediani J.K., Hartman T.J., Pa...","[Health Policy, Health Informatics, Computer S...",Scopus
3,SCOPUS_ID:85169524858,2023-12-01,North American wintering mallards infected wit...,"© 2023, Springer Nature Limited.Avian influenz...",https://www.scopus.com/inward/record.uri?partn...,"[United States Geological Survey, National Wil...","[Teitelbaum C.S., Masto N.M., Sullivan J.D., K...",[Multidisciplinary],Scopus
4,SCOPUS_ID:85169517823,2023-12-01,"Early growth, development and allometry of gly...","© 2023, Springer Nature Limited.This study aim...",https://www.scopus.com/inward/record.uri?partn...,"[University of Georgia, The University of Geor...","[de Souza Rodrigues J., Shilling D., Tishchenk...",[Multidisciplinary],Scopus
...,...,...,...,...,...,...,...,...,...
95,SCOPUS_ID:85161954005,2023-09-01,Neural correlates of autobiographical memory r...,© 2023 Elsevier LtdAutobiographical memory (AM...,https://www.scopus.com/inward/record.uri?partn...,[Emory University],"[Shepardson S., Dahlgren K., Hamann S.]","[Neuropsychology and Physiological Psychology,...",Scopus
96,SCOPUS_ID:85160734595,2023-09-01,Procedure-Specific Center Volume and Mortality...,© 2023 The Society of Thoracic SurgeonsBackgro...,https://www.scopus.com/inward/record.uri?partn...,"[University of Georgia, School of Public Healt...","[Zmora R., Spector L., Bass J., Thomas A., Kni...","[Surgery, Pulmonary and Respiratory Medicine, ...",Scopus
97,SCOPUS_ID:85160300493,2023-09-01,Human Milk Fortification Strategies in the Neo...,None,https://www.scopus.com/inward/record.uri?partn...,[Cincinnati Children's Hospital Medical Center...,"[Fu T.T., Poindexter B.B.]","[Pediatrics, Perinatology and Child Health, Ob...",Scopus
98,SCOPUS_ID:85160066399,2023-09-01,Benign Colorectal Disorders,None,https://www.scopus.com/inward/record.uri?partn...,[Boston University Chobanian &amp; Avedisian S...,"[Aguilar-Alvarado M.Y., Baker B., Chiu L.S., S...",[Pharmacology (medical)],Scopus


In [68]:
df['paper_id'].nunique()

100